# 3. Designing the ln(prior) function

Following the exploration of the existing data we have on parameter values, we are going to design a function that takes the list of parameter values and returns ln(prior) for the whole set, or the sum over all the -ln(priors) for each individual parameter. 

To allow a variation in the form of the distribution and it's shape for each parameter, in addition to the list of parameter values itself (which are going to be randomly selected by the MCMC), we shall pass a dictionary of values for the parameter's distributions, param_dists.

The functions here are now in **inverse/priors.py**.

For an understanding of eg. the form of the ln(prior) function, go to the MCMC Hammer Example notebook.

In [1]:
from scipy import stats
import math
import numpy as np

For now I will define the dictionary by hand. Automated creation to be included in future. 

In [2]:
param_dists = {'tau_e':{'type':'gamma','alpha':2, 'loc':0.004492887509221829, 'scale':0.0032375996670863947},
    'tau_i':{'type':'gamma','alpha':2.006014687419703, 'loc':0.004662441067103153, 'scale':0.0025497764353055712},
              'alpha':{'type':'uniform','lower':0, 'upper':5},
              'speed':{'type':'uniform','lower':0, 'upper':25},
              'gei':{'type':'uniform','lower':0, 'upper':10},
              'gii':{'type':'uniform','lower':0, 'upper':10},
              'tauC':{'type':'gamma','alpha':2, 'loc':0.004211819821836749, 'scale':0.0029499360144432463}}

In [3]:
param_dists['tau_i']['alpha']

2.006014687419703

In [4]:
def lnpriors_test(params, param_dists):
    taui = params[0]
    taui = (taui - param_dists['tau_i']['loc'])/param_dists['tau_i']['scale']
    p = stats.gamma.pdf(taui, param_dists['tau_i']['alpha'])
    return -math.log(p)

In [5]:
params = [0.01, 0.01, 0.010]
lnpriors_test(params, param_dists)

1.3526922972688127

In [1]:
def lnprior_gamma(param, key, param_dists):
    print(key)
    tau = param
    x = (tau - param_dists[key]['loc'])/param_dists[key]['scale']
    print(x, type(x))
    p = stats.gamma.pdf(x, param_dists[key]['alpha'])
    p = p.astype(float)
    print('p = ', p, type(p))
    if p == 0:
        return -np.inf #use the same as for outside uniform distribution bounds
    return math.log(p)

In [25]:
def lnprior_uniform(param, key, param_dists):
    print(key)
    print(param)
    if param_dists[key]['lower'] < param < param_dists[key]['upper']:
        return 0.0
    return -np.inf

We now sum over the parameters, adding together the -ln(prior).

In [26]:
def lnpriors(params, param_dists):
    i = 0
    neg_lnprior = 0
    priorlist = []
    for key in param_dists.keys():
        if param_dists[key]['type'] == 'gamma':
            print(params[i], type(params[i]))
            neg_lnp = lnprior_gamma(params[i], key, param_dists)
        elif param_dists[key]['type'] == 'uniform':
            print(params[i], type(params[i]))
            neg_lnp = lnprior_uniform(params[i], key, param_dists)
        priorlist.append(neg_lnp)
        neg_lnprior += neg_lnp
        i += 1
    return neg_lnprior, priorlist

Test using the default parameter set cast as a list:

In [27]:
params = []
ntf_params = {'tau_e':0.012,
                           'tau_i':0.003,
                           'alpha':1.0,
                           'speed':5.0,
                           'gei':4.0,
                           'gii':1.0,
                           'tauC':0.006
                           }
for key, value in ntf_params.items():
    temp = [key,value]
    params.append(value)

In [28]:
params

[0.012, 0.003, 1.0, 5.0, 4.0, 1.0, 0.006]

In [29]:
neg_lnprior, priorlist = lnpriors(params, param_dists)

0.012 <class 'float'>
tau_e
2.3187278424494124 <class 'float'>
p =  0.22815976974510077 <class 'numpy.float64'>
0.003 <class 'float'>
tau_i
-0.651994835344818 <class 'float'>
p =  0.0 <class 'numpy.float64'>
1.0 <class 'float'>
alpha
1.0
5.0 <class 'float'>
speed
5.0
4.0 <class 'float'>
gei
4.0
1.0 <class 'float'>
gii
1.0
0.006 <class 'float'>
tauC
0.6061759202261008 <class 'float'>
p =  0.3306281470707985 <class 'numpy.float64'>


For now we will use these as a first pass on prior functions (ideally I want to make something a bit more flexible and less hard-coded).

In [30]:
neg_lnprior

-inf

This is -inf because tau_i is outside the range that belongs to the chosen distribution. To check, let's change it to something within the distribution.

In [31]:
params = []
ntf_params = {'tau_e':0.012,
                           'tau_i':0.006,
                           'alpha':1.0,
                           'speed':5.0,
                           'gei':4.0,
                           'gii':1.0,
                           'tauC':0.006
                           }
for key, value in ntf_params.items():
    temp = [key,value]
    params.append(value)

In [32]:
neg_lnprior, priorlist = lnpriors(params, param_dists)

0.012 <class 'float'>
tau_e
2.3187278424494124 <class 'float'>
p =  0.22815976974510077 <class 'numpy.float64'>
0.006 <class 'float'>
tau_i
0.5245789059684957 <class 'float'>
p =  0.308456846520475 <class 'numpy.float64'>
1.0 <class 'float'>
alpha
1.0
5.0 <class 'float'>
speed
5.0
4.0 <class 'float'>
gei
4.0
1.0 <class 'float'>
gii
1.0
0.006 <class 'float'>
tauC
0.6061759202261008 <class 'float'>
p =  0.3306281470707985 <class 'numpy.float64'>


In [33]:
neg_lnprior

3.760643435842928

Let's change them again to make sure things are changing.

In [37]:
params = []
ntf_params = {'tau_e':0.008,
                           'tau_i':0.008,
                           'alpha':1.0,
                           'speed':9.0,
                           'gei':5.0,
                           'gii':1.5,
                           'tauC':0.006
                           }
for key, value in ntf_params.items():
    temp = [key,value]
    params.append(value)

In [38]:
neg_lnprior, priorlist = lnpriors(params, param_dists)

0.008 <class 'float'>
tau_e
1.083244641526146 <class 'float'>
p =  0.36667337757112467 <class 'numpy.float64'>
0.008 <class 'float'>
tau_i
1.3089614001773715 <class 'float'>
p =  0.3532205569009234 <class 'numpy.float64'>
1.0 <class 'float'>
alpha
1.0
9.0 <class 'float'>
speed
9.0
5.0 <class 'float'>
gei
5.0
1.5 <class 'float'>
gii
1.5
0.006 <class 'float'>
tauC
0.6061759202261008 <class 'float'>
p =  0.3306281470707985 <class 'numpy.float64'>


In [39]:
neg_lnprior

3.1507073745035754